In [2]:
from huggingface_hub import snapshot_download

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
snapshot_download(model_id)

Fetching 17 files: 100%|██████████| 17/17 [41:34<00:00, 146.75s/it]


'/llm/cache/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa'

In [2]:
import json
from datasets import Dataset

data = json.load(open("/llm/data/sascha_train.json"))
ds = Dataset.from_list(data)
ds

Generating train split: 128 examples [00:00, 4950.22 examples/s]


Dataset({
    features: ['messages'],
    num_rows: 128
})

In [2]:
def to_messages(sample):
    messages = [{"role": "user", "content": sample["instruction"]}, {"role": "assistant", "content": sample["output"]}]
    sample["messages"] = messages
    return sample
ds = ds.map(to_messages, remove_columns=ds.features)
ds[:3]

Map: 100%|██████████| 128/128 [00:00<00:00, 8609.08 examples/s]


{'messages': [[{'content': 'What is your name?', 'role': 'user'},
   {'content': 'My name is Sascha.', 'role': 'assistant'}],
  [{'content': 'Your name is Sascha. Tell me your name.', 'role': 'user'},
   {'content': 'My name is Sascha.', 'role': 'assistant'}],
  [{'content': 'Tell me your name.', 'role': 'user'},
   {'content': 'Sascha.', 'role': 'assistant'}]]}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

print(tokenizer.apply_chat_template(conversation=ds[0]['messages'], tokenize=False))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

What is your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My name is Sascha.<|eot_id|>


In [4]:
tokenizer.chat_template

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

In [7]:
ds.to_json("/llm/data/sascha_train.jsonl")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 105.62ba/s]


16638

In [3]:
from datasets import load_dataset

ds = load_dataset('json', data_files=["/llm/data/sascha_train.jsonl"])
ds = ds['train']
ds

Dataset({
    features: ['messages'],
    num_rows: 128
})